# Übungsblatt 07
Laden Sie die Daten, bevor Sie mit der Arbeit beginnen.

In [ ]:
%CREATE pg://localhost OF tables.sql

## Aufgabe 1
Eine Hotelkette verwaltet die große Anzahl ihrer Ferienhotels mit einem relationalen Datenbanksystem.

Die Ferienhotels können nur wochenweise gebucht werden. Das Attribut `AnzPers` in Buchung beinhaltet die Anzahl der anreisenden Personen je Buchung. Die Attribute `AnzDZ` und `AnzEZ` geben Auskunft über die Anzahl der vom Gast gewünschten Doppel- bzw. Einzelzimmer. Die Attribute `DZ` und `EZ` in `Hotel` stehen für die Gesamtkapazität an Doppel- bzw. Einzelzimmern eines Hotels.

In [ ]:
%SCHEMA ONLY hotel

Das Hotel möchte auf eine neue Version von PostgreSQL umstellen. Versionen seit 8.4 unterstützen allerdings keine Constraints mehr auf Schema-Level. Schreiben Sie daher die gegebenen Assertions in PostgreSQL kompatible Trigger um.

### Teil a
Schließen Sie Überbelegungen der Zimmer aus. Alle anreisenden Personen je Buchung müssen in den gebuchten Zimmern unterkommen, also mindestens ein Bett haben:

```sql
CREATE ASSERTION Ueberbelegung
CHECK (
    NOT EXISTS (
        SELECT *
        FROM Buchung
        WHERE AnzPers > 2 * AnzDz + AnzEz
    ) )
```

In [ ]:
CREATE FUNCTION pruefe_ueberbelegung()
    RETURNS trigger AS
$func$
BEGIN
    /* TODO
     * ungültige Fälle ausschließen
     */
    RETURN NEW;
END
$func$ LANGUAGE plpgsql;

In [ ]:
CREATE TRIGGER ueberbelegung
BEFORE INSERT ON buchung
FOR EACH ROW
EXECUTE PROCEDURE pruefe_ueberbelegung()

Die nachfolgenden Statements dienen der Prüfung des Triggers.

In [ ]:
INSERT INTO Buchung (GastID, HName, Anreise, AnzPers, AnzDZ, AnzEZ) VALUES
    (1, 'Bergoase Resort', '2024-01-01', 5, 3, 0);

In [ ]:
INSERT INTO Buchung (GastID, HName, Anreise, AnzPers, AnzDZ, AnzEZ) VALUES
    (2, 'Grüne Oase Hotel', '2024-02-01', 6, 2, 1);

### Teil b
Achten Sie darauf, dass für alle Anreisetermine die Hotelkapazitäten nicht überschritten werden, d.h. weder zu viele Einzelzimmer noch zu viele Doppelzimmer vergeben sind.

```sql
CREATE ASSERTION Ueberbuchung
CHECK (
    NOT EXISTS (
        SELECT hotel.HName
        FROM buchung, hotel
        WHERE buchung.HName = hotel.HName
        GROUP BY hotel.HName, Anreise, hotel.DZ, hotel.EZ
        HAVING SUM(buchung.AnzDz) > hotel.DZ OR SUM(buchung.AnzEz) > hotel.EZ
    )
)
```

In [ ]:
CREATE FUNCTION pruefe_ueberbuchung()
    RETURNS trigger AS
$func$
BEGIN
    /* TODO
     * ungültige Fälle ausschließen
     */
    RETURN NEW;
END
$func$ LANGUAGE plpgsql;

In [ ]:
CREATE TRIGGER ueberbuchung
BEFORE INSERT ON buchung
FOR EACH ROW
EXECUTE PROCEDURE pruefe_ueberbuchung()

Die nachfolgenden Statements dienen der Prüfung des Triggers.

In [ ]:
INSERT INTO Buchung (GastID, HName, Anreise, AnzPers, AnzDZ, AnzEZ) VALUES
    (11, 'Stadtpalais Hotel', '2024-02-01', 5, 3, 3)

In [ ]:
INSERT INTO Buchung (GastID, HName, Anreise, AnzPers, AnzDZ, AnzEZ) VALUES
    (12, 'Stadtpalais Hotel', '2024-03-01', 2, 1, 0)

In [ ]:
INSERT INTO Buchung (GastID, HName, Anreise, AnzPers, AnzDZ, AnzEZ) VALUES
    (13, 'Alpenblick Lodge', '2024-04-01', 5, 2, 3)

In [ ]:
INSERT INTO Buchung (GastID, HName, Anreise, AnzPers, AnzDZ, AnzEZ) VALUES
    (14, 'Alpenblick Lodge', '2024-05-01', 3, 0, 3)

## Aufgabe 2
Formulieren Sie PostgreSQL kompatible Trigger für folgende Bedingungen:

### Teil a
Nachdem Gäste abgereist sind, wird ihre Buchung gelöscht. Formulieren Sie einen Trigger, der dafür sorgt, dass auch der entsprechende Gasteintrag gelöscht wird, wenn dieser keine weiteren Buchungen hat.

In [ ]:
CREATE FUNCTION gast_loeschen()
    RETURNS trigger AS
$func$
BEGIN
    /* TODO
     * Gast löschen, falls keine weiteren Buchungen
     * vorhanden sind
     */
    RETURN NEW;
END
$func$ LANGUAGE plpgsql;

In [ ]:
CREATE TRIGGER gast_loeschen
AFTER DELETE ON buchung
FOR EACH ROW
EXECUTE PROCEDURE gast_loeschen()

Die nachfolgenden Statements dienen der Prüfung des Triggers.

In [ ]:
-- Gast 7 ist vorhanden
SELECT * FROM Gast WHERE GastID = 7

In [ ]:
-- Alle Buchungen von Gast 7 werden gelöscht
DELETE FROM Buchung WHERE GastID = 7

In [ ]:
-- Gast 7 sollte vom Trigger gelöscht sein
SELECT * FROM Gast WHERE GastID = 7

### Teil b
Umbuchungen des Hotels werden nur akzeptiert, wenn der Grund dafür eine geringere Entfernung zum Strand ist.

In [ ]:
CREATE FUNCTION umbuchung_pruefen()
    RETURNS trigger AS
$func$
BEGIN
    /* TODO
     * Änderung ablehnen, falls sich Nähe zum
     * Strand nicht verringert
     */
    RETURN NEW;
END
$func$ LANGUAGE plpgsql;

In [ ]:
CREATE TRIGGER umbuchung_pruefen
BEFORE UPDATE ON buchung
FOR EACH ROW
EXECUTE PROCEDURE umbuchung_pruefen()

Die nachfolgenden Statements dienen der Prüfung des Triggers.

In [ ]:
-- Gast 4: Strandperle Hotel (5km) -> Seeblick Hotel (2km)
UPDATE Buchung
SET HName = 'Seeblick Hotel'
WHERE GastID = 4 AND HName = 'Strandperle Hotel'

In [ ]:
-- Gast 6: Strandperle Hotel (5km) -> Panorama Hotel (8km)
UPDATE Buchung
SET HName = 'Panorama Hotel'
WHERE GastID = 6 AND HName = 'Strandperle Hotel'

## Aufgabe 3
Es sind verschiedene Sichten auf die Hotel-Datenbank für das Management (Sicht `V1`) bzw. das Personal des *Strandperle Hotel* (Sicht `V2`) definiert:

In [ ]:
CREATE OR REPLACE VIEW V1 (Betrieb, Kapazität) AS (
    SELECT HName, DZ * 2 + EZ
    FROM Hotel
)

In [ ]:
SELECT * FROM V1

In [ ]:
CREATE OR REPLACE VIEW V2 AS (
    SELECT *
    FROM Buchung
    WHERE HName = 'Strandperle Hotel'
)

In [ ]:
SELECT * FROM V2

In der täglichen Arbeit ergeben sich die nachfolgenden Änderungen. Diskutieren Sie die Probleme und Konsequenzen der SQL-Anweisungen anhand der Kriterien für Änderungen auf Sichten und beurteilen Sie deren Zulässigkeit.

Nach einem Umbau hat sich die Kapazität des *Seeblick Hotel* um 10 (Betten) erhöht.

In [ ]:
UPDATE V1
SET Kapazität = Kapazität + 10
WHERE Betrieb = 'Seeblick Hotel'

Die Buchung des Gastes $3$ im *Strandperle Hotel* wird storniert.

In [ ]:
DELETE FROM V2
WHERE GastID = 3

Der Gast $6$ soll vom *Strandperle Hotel* ins *Küstenparadies Resort* umgebucht werden.

In [ ]:
UPDATE V2
SET HName = 'Küstenparadies Resort'
WHERE GastID = 6

## Aufgabe 4
Ein Auslieferungslager wird durch eine Datenbank mit folgendem Schema unterstützt:

In [ ]:
%SCHEMA ONLY lager

Das Auslieferungslager liefert Teile aus und bestellt vom Hersteller nach. Die nachfolgenden Änderungen werden auf den Relationen ausgeführt:

In [ ]:
-- 1

INSERT INTO Auslieferung VALUES (739, 4713, 5, 225.00);

-- 2

UPDATE Lager SET Vorrat = Vorrat - 5 WHERE TeilNr = 4713;

-- 3

INSERT INTO Nachbestellung VALUES (4713, 5, 'Siemens');

-- 4

### Teil a
Bestimmen Sie für die Änderungen Transaktionen im Sinne des ACID-Prinzips. Wo muss ein `COMMIT` zur Kennzeichnung des Endes einer Transaktion in die Befehlsfolg eingefügt werden? **Begründen** Sie kurz Ihre Entscheidung

### Teil b
Welche Konsistenzprobleme können bei den Änderungen in a) auftreten und wie können diese vermieden werden?